pip install -i https://pypi.yandex-team.ru/simple/ nile yql

In [9]:
from yql.api.v1.client import YqlClient
from nile.api.v1 import clusters, Record, filters as nf, aggregators as na
from operator import attrgetter

In [10]:
login = "ar7is7"
yql_token = ""
date_start = "2019-10-25"
date_end = "2019-10-28"


yt_tmp_folder = "//tmp/{}".format(login)

template = {
    'tmp_folder': yt_tmp_folder,
    'date_start': date_start,
    'date_end': date_end
}

In [11]:
def run_yql(query):
    client = YqlClient(token=yql_token)
    request = client.query(query)
    request.run()
    results = request.get_results()

    result_df = results.dataframe
    return result_df

In [ ]:
#YQL v0
run_yql("""
USE hahn;
pragma simplecolumns;
insert into [{tmp_folder}/new_taxi_vins] with truncate
SELECT a.*, Yson::ConvertToString(Yson::SerializePretty(a.request.request_id))  as request_id
FROM range([//home/voice/vins/logs/dialogs], [{date_start}], [{date_end}]) as a
""".format(**template))

In [12]:
def mapper(records):
    for rec in records:
#         if int(rec['session_id'].split("_")[1]) <= 1570798800: continue // filter by time
        for el in rec['session']:
            if el['intent'] and "taxi_new" in el['intent']:
                yield rec
                break
                
                
def mapper1(records):
    for record in records:
        n = len(record['session'])
        first, last = -1, n
        for el in range(n):
            if "taxi_new" in record['session'][el]['intent'] or "taxi_new" in record['session'][el].get('restored', ''):
                if first == -1:
                    first = el
                last = el
        first = max(first-3, 0)
        last = min(last+4, n)
        yield Record(record, session=record['session'][first:last])
            

            
cluster = clusters.YT(proxy="hahn.yt.yandex.net").env(
    templates={"date": "{{{date_start}..{date_end}}}".format(**template)}
)
job = cluster.job()
job.table('//home/voice/dialog/sessions/@date') \
        .map(mapper) \
        .map(mapper1) \
        .put('{tmp_folder}/new_taxi_afted_16_00'.format(**template))

job.run()

VBox()

In [13]:
def mapper(records):
    for rec in records:
        for idx, el in enumerate(rec['session']):
            yield Record.from_dict({
                "request_id": el["req_id"],
                "idx": idx,
                "session_id": rec["session_id"]
            })
            
cluster = clusters.YT(proxy="hahn.yt.yandex.net")
job = cluster.job()
job.table('{tmp_folder}/new_taxi_afted_16_00'.format(**template)) \
        .map(mapper) \
        .put('{tmp_folder}/new_taxi_afted_16_00_splitted'.format(**template))

job.run()

VBox()

In [ ]:
run_yql("""
USE hahn; 
PRAGMA yt.InferSchema;
PRAGMA SimpleColumns;
PRAGMA yt.Pool = "voice";
insert into [{tmp_folder}/new_taxi_req_with_audio]
WITH TRUNCATE 
SELECT some(mds_key) as mds_key, idx, request_id, session_id
FROM (
    SELECT a.*, b.mds_key as mds_key
    FROM [{tmp_folder}/new_taxi_afted_16_00_splitted] as a
    LEFT JOIN range([//home/voice-speechbase/uniproxy/logs_unified_qloud], [{date_start}], [{date_end}]) as b
    ON a.request_id == b.vins_request_id
)as a
GROUP BY idx, request_id, session_id
""".format(**template)
)

Traceback (most recent call last):
  File "/home/ar7is7/anaconda3/envs/py27/lib/python2.7/site-packages/yql/client/request.py", line 110, in run
    self.result.raise_for_status()
  File "/home/ar7is7/.local/lib/python2.7/site-packages/requests/models.py", line 937, in raise_for_status
    raise HTTPError(http_error_msg, response=self)
HTTPError: 404 Client Error: Not Found for url: https://yql.yandex.net/api/v2/operations/5db802d1095c4e79fce66416/results?filters=DATA


In [ ]:
run_yql("""
USE hahn; 
PRAGMA yt.InferSchema;
PRAGMA SimpleColumns;
PRAGMA yt.Pool = "voice";
insert into [{tmp_folder}/new_taxi_with_response]
WITH TRUNCATE 
SELECT some(response) as response, idx, request_id, session_id
FROM (
    SELECT a.idx as idx, a.request_id as request_id, a.session_id as session_id, b.response as response
    FROM [{tmp_folder}/new_taxi_afted_16_00_splitted] as a
    LEFT JOIN [{tmp_folder}/new_taxi_vins] as b
    ON a.request_id == b.request_id
) as a
GROUP BY idx, request_id, session_id
""".format(**template)
)

In [ ]:
def mapper(records):
    for record in records:
        rec = record.to_dict()
        el = {}
        for meta in rec['response']['meta']:
            if meta['type'] == "analytics_info" and meta.get('form', {}).get('slots'):
                for slot in meta['form']['slots']:
                    if slot and slot.get('value_type') == "geo" and slot['value'] and slot['value'].get('address_line'):
                        el[slot['slot']] = slot['value']['address_line']
                    if slot and slot.get('value_type') == "poi":
                        if slot['value'] and slot['value'].get('company_name'):
                            el[slot['slot']] = slot['value']['company_name']
                        if slot['value'] and slot['value'].get('geo').get('address_line'):
                            el[slot['slot']] += " по адресу: " + slot['value']['geo']['address_line']
        yield Record.from_dict({
            "request_id": rec['request_id'], 
            "idx": rec["idx"], 
            "session_id": rec["session_id"], 
            "geo":el
        })

cluster = clusters.YT(proxy="hahn.yt.yandex.net")
job = cluster.job()
us = job.table('{tmp_folder}/new_taxi_with_response'.format(**template)) \
        .map(mapper) \
        .put('{tmp_folder}/new_taxi_with_geo'.format(**template))

job.run()

In [ ]:
run_yql("""
USE hahn; 
PRAGMA yt.InferSchema;
PRAGMA SimpleColumns;
PRAGMA yt.Pool = "voice";
insert into [{tmp_folder}/new_taxi_with_audio_geo]
WITH TRUNCATE 
SELECT a.mds_key as mds_key, a.idx as idx, a.request_id as request_id, a.session_id as session_id, b.geo as geo
FROM [{tmp_folder}/new_taxi_req_with_audio] as a
join [{tmp_folder}/new_taxi_with_geo] as b
USING (request_id)
""".format(**template)
)

In [ ]:
def mapper(records):
    for record in records:
        rec = record.to_dict()
        if '_other' in rec:
            del rec['_other']
        res = []
        for ses, aud in zip(rec['session'], rec['column']):
            ses['geo'] = aud['geo']
            ses['mds_key'] = aud['mds_key']
            res.append(ses)
        rec['session'] = res
        del rec['column']
        yield Record(**rec)
        
def reducer(sessions):
    for _, records in sessions:
        
        t1 = list(map(lambda x: {
            "idx": x["idx"],
            "geo": x["geo"],
            "mds_key": x["mds_key"],
            "request_id": x["request_id"],
            "session_id": x["session_id"]
        }, sorted(list(records), key=attrgetter('session_id', 'idx'))))
        t2 = [{"session_id": "session_id"}] + t1 + [{"session_id": "session_id"}]
        ans, cur = [], []
        for idx in range(1, len(t2)):
            if t2[idx]['session_id'] == t2[idx-1]['session_id']:
                cur.append(t2[idx])
            else:
                if cur:
                    yield Record.from_dict({
                        "column": cur,
                        "session_id": cur[0]["session_id"]
                    })
                cur = [t2[idx]]
        

cluster = clusters.YT(proxy="hahn.yt.yandex.net")
job = cluster.job()
for_join = job.table("{tmp_folder}/new_taxi_with_audio_geo".format(**template)) \
        .groupby('session_id') \
        .reduce(reducer)


job.table('{tmp_folder}/new_taxi_afted_16_00'.format(**template)) \
        .join(for_join, by='session_id') \
        .map(mapper) \
        .put('{tmp_folder}/new_taxi_with_all'.format(**template))

job.run()

In [ ]:
def mapper2(records, output1, output2):
    for record in records:
        flag = True
        for el in record['session']:
            if el['intent'] and "taxi_new_order" in el['intent'] and "confirm" in el['intent'] and "confirmation_no" not in el['intent']:
                output1(record)
                flag = False
                continue
        if flag:
            output2(record)
            

cluster = clusters.YT(proxy="hahn.yt.yandex.net")
job = cluster.job()
o1, o2 = job.table('{tmp_folder}/new_taxi_with_all'.format(**template)) \
        .map(mapper2)
o1.put('{tmp_folder}/new_taxi_with_confirm'.format(**template))
o2.put('{tmp_folder}/new_taxi_without_confirm'.format(**template))

job.run()

In [ ]:
https://nirvana.yandex-team.ru/flow/29d49660-b250-493d-b622-34c611233455/af972b7a-1b9a-400d-ab4f-baba1877978d/graph
229163 - created pool in toloka for uploading task suites

In [ ]:
https://nirvana.yandex-team.ru/flow/29d49660-b250-493d-b622-34c611233455/d1eb6c6e-90d3-469f-8d3a-7b2a9784973a/graph
229164 - created pool in toloka for uploading task suites